# BIP0322 Verification

Initially this is just attempting to verify the test vectors provided in BIP0322

### Simple Signature


From the BIP-0322:

`A simple signature consists of a witness stack, consensus encoded as a vector of vectors of bytes, and base64-encoded. Validators should construct to_spend and to_sign as defined below, with default values for all fields except that`

### Test Vector Sig

This signature is the test vector taken from [bip0322](https://github.com/bitcoin/bips/blob/master/bip-0322.mediawiki#message-signing).

Currently unable to get this to verify.

In [2]:
simple_signature = 'AkcwRAIgG3PASL/vRTgAqogWT6S8rUOQXNnfRzX6JncmbFlHc1ACIGQdsW+rnVmsQzyAYRQisHKFMigDmKiL7LUw4x17Fw5tASECx/EgAxlkQpQ9hYjgGu6EBCPMVPwVIVJqO4XCsMvViHI='

### Produced Sig

This signature was produced in the BIP0322_signing notebook. It seems to be the one that verifie against the message and signing address.

**Note: Perhaps I am just replicating the same mistake across both notebooks?**

In [3]:
produced_sig = 'AkcwRAIgWPoSeW/VClUS8rjapPqeJwQNPbPZgMNP5I6vTd054RwCIF0lCZyYNU/E6mtuGa/pXwyGMwu1ih5TI3S0uLtNhexKASECx/EgAxlkQpQ9hYjgGu6EBCPMVPwVIVJqO4XCsMvViHI='

### Signing Address

In [4]:
address = 'bc1q9vza2e8x573nczrlzms0wvx3gsqjx7vavgkx0l'

### Message Signed

In [18]:
from buidl.hash import tagged_hash
from buidl.helper import hash256
message = b"Hello World"
tag = b"BIP0322-signed-message"

tagged_hash = tagged_hash(tag,message)

test_vector = 'f0eb03b1a75ac6d9847f55c624a99169b5dccba2a31f5b23bea77ba270de0a7a'
assert(test_vector == tagged_hash.hex())

# Verification Process

## 1. Basic Validation

### 1a. Compute virtual to_spend transaction
```
    nVersion = 0
    nLockTime = 0
    vin[0].prevout.hash = 0000...000
    vin[0].prevout.n = 0xFFFFFFFF
    vin[0].nSequence = 0
    vin[0].scriptSig = OP_0 PUSH32[ message_hash ]
    vin[0].scriptWitness = []
    vout[0].nValue = 0
    vout[0].scriptPubKey = message_challenge
```

In [21]:
from buidl.tx import Tx, TxIn, TxOut
from buidl.helper import little_endian_to_int
from buidl.script import Script

In [22]:
# Not a valid Tx hash. Will never be spendable on any BTC network.
prevout_hash = bytes.fromhex('0000000000000000000000000000000000000000000000000000000000000000')
# prevout.n
prevout_index = little_endian_to_int(bytes.fromhex('FFFFFFFF'))
sequence = 0

# Byte array of message hash
message_hash = tagged_hash
commands = [0, 32, message_hash]
scriptSig = Script(commands)
print(scriptSig)
# Create Tx Input
tx_in = TxIn(prevout_hash,prevout_index,scriptSig,sequence)

OP_0 OP_[32] 42a873ac3abd02122d27e80486c6fa1ef78694e8505fcec9cbcc8a7728ba8949 


In [23]:
tx_in

0000000000000000000000000000000000000000000000000000000000000000:4294967295

In [24]:
from buidl.script import address_to_script_pubkey

# Value of tx output
value = 0

script_pubkey = address_to_script_pubkey(address)

print(script_pubkey)
tx_out = TxOut(value,script_pubkey)

OP_0 2b05d564e6a7a33c087f16e0f730d1440123799d 


In [25]:
# create transaction
version=0
tx_inputs = [tx_in]
tx_outputs = [tx_out]
locktime=0
network="mainnet"

# Could be false, but using a segwit address. I think this is the "Simple Signature" in BIP-0322
segwit=True

virtual_to_spend_tx = Tx(version,tx_inputs,tx_outputs,locktime,network,segwit)

In [26]:
print(virtual_to_spend_tx)


tx: 6274d1558a583d7e7c11df9c939bb9e96ef543cc7f5df8f78df0b6ccdb05c690
version: 0
locktime: 0
tx_ins:
0000000000000000000000000000000000000000000000000000000000000000:4294967295
tx_outs:
0:OP_0 2b05d564e6a7a33c087f16e0f730d1440123799d 



### 1b.Decode the signature as the transaction virtual to_sign

Note: the signature in simple format is just the witness stack

```python
    nVersion = 0 or (FULL format only) as appropriate (e.g. 2, for time locks)
    nLockTime = 0 or (FULL format only) as appropriate (for time locks)
    vin[0].prevout.hash = to_spend.txid
    vin[0].prevout.n = 0
    vin[0].nSequence = 0 or (FULL format only) as appropriate (for time locks)
    vin[0].scriptWitness = message_signature
    vout[0].nValue = 0
    vout[0].scriptPubKey = OP_RETURN
```

#### Decode signature into witness stack

In [33]:
from buidl.helper import base64_decode
from buidl.ecc import Signature
from buidl.witness import Witness

# Note: Using the test vector I am unable to get the code to verify
decoded_tx = base64_decode(simple_signature)

# decoded_tx = base64_decode(produced_sig)

import io
stream = io.BytesIO(decoded_tx)
witness_stack = Witness.parse(stream)
print(witness_stack)

304402201b73c048bfef453800aa88164fa4bcad43905cd9df4735fa2677266c594773500220641db16fab9d59ac433c80611422b0728532280398a88becb530e31d7b170e6d01 02c7f12003196442943d8588e01aee840423cc54fc1521526a3b85c2b0cbd58872 


#### Create tx_in

In [34]:
prevout_hash = virtual_to_spend_tx.hash()
prevout_index = 0
sequence = 0
witness = witness_stack

tx_in = TxIn(prevout_hash,prevout_index,sequence=sequence)
tx_in.witness = witness

tx_in.witness

304402201b73c048bfef453800aa88164fa4bcad43905cd9df4735fa2677266c594773500220641db16fab9d59ac433c80611422b0728532280398a88becb530e31d7b170e6d01 02c7f12003196442943d8588e01aee840423cc54fc1521526a3b85c2b0cbd58872 

#### Create tx_out

In [35]:
value = 0
# OP Code 106 for OP_RETURN
commands = [106]
scriptPubKey = Script(commands)

tx_output = TxOut(value,scriptPubKey)

#### Create virtual_to_sign_tx

In [36]:
version = 0
locktime = 0
virtual_to_sign_tx = Tx(version, [tx_in], [tx_output],locktime,segwit=False)

### A whole bunch of validation that SHOULD happen ...

### Initially all I am trying to do is validate the signature on the input in the to_sign transaction


### The input for the virtual to_sign transation IS the output at index 0 from the virtual to_spend transaction

Normally the relevant information (The outputs script_pubkey and value) would be populated by querying the network for the transaction and fetching it from there. In this case we set those values on the input manually.

In [37]:
virtual_to_sign_tx.tx_ins[0]._script_pubkey = script_pubkey
virtual_to_sign_tx.tx_ins[0]._value = 0

### Verify the input is a valid unlocking of the to_spend output

Not sure why this isn't working for the test vector of BIP-0322

In [38]:
virtual_to_sign_tx.verify_input(0)

False